In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import requests
import re
from bs4 import BeautifulSoup
import json
import urllib

In [2]:
transp_reg = gpd.read_file('./data/from/transp_reg_st.shp',
                        encoding='utf-8')
# transp_reg.crs = 'epsg:53004'
transp_reg = transp_reg.to_crs('epsg:4326')

In [3]:
def find_js(lnk):
    with requests.get(lnk, stream=True, timeout=25) as req:
        bs = BeautifulSoup(req.text, 'html.parser')
        all_scrpt = bs.find_all("script")
        for scr in all_scrpt:
            if scr.get('type') == "application/json":
                sc_ind = all_scrpt.index(scr)
                break
            else:
                sc_ind=0
        # 
        scrp_txt = all_scrpt[sc_ind].text
        js = json.loads(scrp_txt)

        return js

In [4]:
def get_poi(word, srch_pt, poly):

    str_qt = urllib.parse.quote(word)

    coords1 = str(str(srch_pt.coords[0][0]) + "%2C" + str(srch_pt.coords[0][1]))
    coords2 = coords1

    url = "https://yandex.ru/maps/51/samara/search/{str_qt}/?ll={coords1}&sll={coords2}&sspn=0.015134%2C0.006711&z=15.96".format(str_qt=str_qt,
                                                                                                                                       coords1=coords1,
                                                                                                                                       coords2=coords2)
    # 
    js = find_js(url)

    fnd_items = js['searchPreloadedResults']['items']

    if len(fnd_items) == 0:
        lst_res = [word+'_not_found', None, srch_pt]
    else:
        i=0
        for i in range(len(fnd_items)):
            fnd_pt = Point(fnd_items[i]['coordinates'])
            if poly.intersects(fnd_pt):
                name = fnd_items[0]['title']
                lst_res = [word, name, fnd_pt]
                break
            else:
                lst_res = [word+'_not_found', None, srch_pt]
    # 
    return lst_res

In [5]:
def get_reg_centre(reg, lst_words):

    poly = reg.geometry[0]
    srch_pt = poly.centroid
    reg_id = reg.NO[0]

    bg_lst_found = []
    i=0
    for i in range(len(lst_words)):
        word = lst_words[i]
        try:
            lst_res = [reg_id] + get_poi(word, srch_pt, poly)
        except:
            time.sleep(30)
            lst_res = [reg_id] + get_poi(word, srch_pt, poly)
        bg_lst_found.append(lst_res)
    # 
    return bg_lst_found

In [6]:
def get_dist_n_time(js, tp_ts):

    if tp_ts == 'auto':
        distnc = round(js['routerResponse']['routes'][0]['distance']['value'] / 1000, 2)
        tm_avg = round(js['routerResponse']['routes'][0]['duration'] / 60, 1)
        tm_trfc = round(js['routerResponse']['routes'][0]['durationInTraffic'] / 60, 1)
    else:
        tm_avg = round(js['routerResponse']['routes'][0]['duration'] / 60, 1)
        distnc = None
        tm_trfc = None
    # 
    lst_dist_time = [distnc, tm_avg, tm_trfc]

    return lst_dist_time

In [7]:
def make_str_coords(one_reg_pt, two_reg_pt):
    coord_str = str(str(one_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(one_reg_pt.coords[0][0]) 
                    + "~" 
                    + str(two_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(two_reg_pt.coords[0][0]))
    # 
    return coord_str

In [8]:
lst_words = ['Торговый центр',
                'Железнодорожная станция',
                'Автостанция',
                'Площадь',
                'Больница',
                'ВУЗ',
                'Завод']

In [216]:
# bg_lst_pts = []

# i=0
# # for i in tqdm(range(2)):
# for i in tqdm(range(len(transp_reg))):
#     reg = transp_reg.iloc[[i]].reset_index(drop=True)
#     bg_lst_found = get_reg_centre(reg, lst_words)
#     bg_lst_pts = bg_lst_pts + bg_lst_found
# # 

In [217]:
# clmns = ['reg_id', 'grp_pt', 'name_pt', 'geometry']
# df = pd.DataFrame(bg_lst_pts, columns = clmns)
# gdf_pts = gpd.GeoDataFrame(df, geometry='geometry')
# gdf_pts.crs='epsg:4326'

In [218]:
# gdf_pts.to_file('./data/res/gdf_pts_all.json', driver='GeoJSON', encoding='utf-8')

In [9]:
gdf_pts_select = gpd.read_file('./data/res/gdf_pts_all_select.geojson',
                        encoding='utf-8')

In [10]:
def get_rt_data(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to):

    coord_str = make_str_coords(one_reg_pt, two_reg_pt)
    str_coord = str(one_reg_pt.coords[0][1]) + "%2C"  +  str(one_reg_pt.coords[0][0])
    
    # tp_ts_mt="mt"
    url_mt = "https://yandex.ru/maps/51/samara/?ll={}&mode=routes&rtext={}&rtt=mt&ruri=~&z=14.41". format(str_coord, coord_str)
    # tp_ts_auto = "auto"
    url_auto = "https://yandex.ru/maps/51/samara/?ll={}&mode=routes&rtext={}&rtt=auto&ruri=~&z=14.41". format(str_coord, coord_str)
    
    js_mt = find_js(url_mt)
    js_auto = find_js(url_auto)
    lst_dist_time_auto = get_dist_n_time(js_auto, 'auto')
    try:
        lst_dist_time_mt = get_dist_n_time(js_mt, 'mt')
    except:
        lst_dist_time_mt = [None, None, None]
    x_from = one_reg_pt.x
    y_from = one_reg_pt.y
    x_to = two_reg_pt.x
    y_to = two_reg_pt.y
    
    line_geo = LineString(js_auto['routerResponse']['routes'][0]['coordinates'])
    
    lst_one_data = [[reg_id_from, reg_id_to] 
                    + lst_dist_time_auto 
                    + [lst_dist_time_mt[1]] + [x_from, y_from, x_to, y_to] + [line_geo]]
    #
    return lst_one_data

In [11]:
lst_from = list(gdf_pts_select.reg_id.unique())
lst_to = list(gdf_pts_select.reg_id.unique())

In [323]:

lst_total_data=[]
lst_done=[]


j=0
for j in tqdm(range(len(lst_from))):

    i=0
    for i in tqdm(range(lst_to)):
    
        reg_id_from, reg_id_to = lst_from[i], lst_to[j]
        
        dnpr = str(reg_id_from) + "_" + str(reg_id_to)
        if dnpr not in lst_done:
            lst_done.append(dnpr)
            if reg_id_from != reg_id_to:

                one_reg_pt = gdf_pts_select[gdf_pts_select.reg_id 
                                            == reg_id_from].reset_index(drop=True).geometry[0]
                two_reg_pt = gdf_pts_select[gdf_pts_select.reg_id 
                                            == reg_id_to].reset_index(drop=True).geometry[0]
                try:
                    lst_one_data = get_rt_data(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to)
                    lst_total_data = lst_total_data + lst_one_data
                except:
                    time.sleep(30)
                    try:
                        lst_one_data = get_rt_data(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to)
                        lst_total_data = lst_total_data + lst_one_data
                    except:
                        print(reg_id_from,reg_id_to)
                        pass
                try:
                    lst_one_data2 = get_rt_data(two_reg_pt, one_reg_pt, reg_id_to, reg_id_from)
                    lst_total_data = lst_total_data + lst_one_data2
                except:
                    time.sleep(30)
                    try:
                        lst_one_data2 = get_rt_data(two_reg_pt, one_reg_pt, reg_id_to, reg_id_from)
                        lst_total_data = lst_total_data + lst_one_data2
                    except:
                        print(reg_id_to, reg_id_from)
                        pass

        #
# 

161 532
532 162
162 532
532 163
163 532
532 164
164 532
532 165
165 532
532 166
166 532
532 229
229 532
532 278
278 532
532 279
279 532
532 299
299 532
532 300
300 532
532 301
301 532
532 302
302 532
532 303
303 532
532 305
305 532
532 308
308 532
532 309
309 532
532 310
310 532
532 485
485 532
532 509
509 532
532 510
510 532
532 511
511 532
532 512
512 532
532 513
513 532
532 514
514 532
532 515
515 532
532 516
516 532
532 517
517 532
532 518
518 532
532 519
519 532
532 522
522 532
532 523
523 532
532 524
524 532




In [324]:
clmns=['id_reg_from', 'id_reg_to', 'length', 'time_auto_avg', 
       'time_auto_traff', 'time_pubtr', 'x_from', 'y_from', 'x_to', 'y_to', 'geometry']
df_rt = pd.DataFrame(lst_total_data, columns = clmns)
gdf_rt = gpd.GeoDataFrame(df_rt, geometry='geometry')

gdf_rt = gdf_rt.reset_index(drop=True).reset_index()

gdf_rt.crs='epsg:4326'


In [325]:
gdf_rt.to_file('./data/res/gdf_rt_new_prt5.json', driver='GeoJSON', encoding='utf-8')

In [ ]:
# откуда, куда, расстояние, авто_время среднее, авто_время в пробках, 
# общтр_время среднее, точко откуда, точка куда

In [260]:
# lst_done2 = list(gdf_rt['id_reg_from'].astype(str) + "_" + gdf_rt['id_reg_to'].astype(str))
# lst_done3 = lst_done + lst_done2 

In [16]:
gdf_rt_all3 = gpd.read_file('./data/res/gdf_rt_all3.json',encoding='utf-8')

In [288]:
# no_geo_gdf_rt_all = gdf_rt_all.copy()
# del no_geo_gdf_rt_all['geometry']
# no_geo_gdf_rt_all.to_csv('./data/res/gdf_rt_all_csv.csv', sep=';', encoding='utf-8')

In [17]:
lst_all = list(gdf_rt_all3['id_reg_from'].astype(str) 
               + "_" 
               + gdf_rt_all3['id_reg_to'].astype(str))

In [12]:
lst_pts=[]
lst_one=list(gdf_pts_select['reg_id'].astype(str))

i=0
for i in range(len(lst_one)):
    j=0
    for j in range(len(lst_one)):
        one_pair = lst_one[i] + "_" + lst_one[j]
        if one_pair not in lst_pts:
            lst_pts.append(one_pair)
# 

In [14]:
len(lst_pts)

13225

In [18]:
lst_left = []
for k in range(len(lst_pts)):
    one = lst_pts[k]
    if one not in lst_all:
        lst_left.append(one)

In [19]:
len(lst_left)

115